# Programowanie obiektowe 2 - decydujące starcie

Zacznijmy od krótkiego, hasłowego, niepełnego podsumowania pierwszej części materiałów.

### W poprzednim odcinku ...
1. Atrybuty
- Można dodać nowy atrybut do już utworzonego obiektu - Rada: nie robimy tego. Inicjalizujmy atrybuty (obiektowe) w metodzie `__init__`
2. Metody
- przypomnijmy, w Pythonie funkcje to obiekty pierwszej klasy
_(tworzone w czasie wykonania, przypisywane do zmiennej lub elementu w strukturze danych, przekazywane jako argument do funkcji, zwracane jako wynik funkcji)_
-- metody w klasach to po prostu atrybuty, których wartościami są funkcje
-- funkcja (z parametrem będącym obiektem) i metoda z perspektywy Pythona wyglądają tak samo
-- Rada: w prostych interfejsach akceptuj funkcje zamiast klas (zob też 1.2.1.)
- bound method - metoda powiązana z obiektem
- w Pythonie nie możemy przeciążać metod
3. Porównywanie `is` vs `==`
4. Widoczność
- dziurawa hermetyzacja - zgodnie z mottem "jesteśmy dorośli" - język nie uniemożliwi nam dostępu nawet do `__atrybutów`
5. Konstruktory
- Rada: korzystaj z `super().__init__()` zamiast `MojaNadklasa.__init__()`, aby w przypadku wielodziedziczenia mieć pewność, że metody nadklas wykonają się tylko raz (zob. też 2)

Zaraz, zaraz, jak to wielodziedziczenia ???

### Polecana literatura
1. Zaawansowany Python (Fluent Python), Ramalho
2. Efektywny Python, Slatkin
3. Python Distilled, Beazley

Tymczasem w niniejszych materiałach zmierzymy się z:
### Zagadnienia
1. Pythoniczność obiektów
2. Wielokrotne dziedziczenie
3. Method resolution order - MRO
4. Klasy domieszki
5. Różne uwagi
6. Metaklasy

## 1. Pythoniczność obiektów

Istnieje kilka nadrzędnych zasad pisania dobrego oprogramowania, które można debugować, testować i rozszerzać. Wzorce projektowe znane z innych języków możesz oczywiście zastosować w Pythonie. Jednak wiele z nich rozwiązuje problemy wynikające ze ścisłej natury systemu statycznych typów C++ lub Javy. W Pythonie mogą być przesadne lub niepotrzebne. Ogólna zasada pozostaje taka sama, sposób programowania może być inny.

### 1.1 Uniwersalność obiektów poprzez definiowanie metod dunder

Pythoniczność - typy definiowane przez użytkownika mogą działać tak naturalnie jak typy wbudowane. Możemy osiągnąć to bez dziedziczenia w duchu kaczego typowania.


#### 1.1.1. Film v.0
Zobaczmy przykład z klasą film i znanymi już `__repr__`, `__str__`, `__eq__`, `__lt__`. Przyjrzyjmy się `__iter__` i `__len__`. Dzięki `__iter__` możemy definiować jak iterować po naszym obiekcie. Możemy potraktować nasze atrybuty jako kolejne elementy, z których możemy zrobić krotkę...

In [104]:
class Film:
    def __init__(self, tytuł, ocena, długość):
        self.tytuł = tytuł
        self.ocena = ocena
        self.długość = długość

    def __repr__(self):
        return "Film:"+str(self)

    def __str__(self):
        return str(tuple(self))         # zdefiniujemy teraz __iter__ aby można było robić tuple(Film)

    def __iter__(self):
        return iter((self.tytuł, self.ocena, self.długość))             # tu podajemy nasz iterator
        # wygenerowany za pomocą funkcji wbudowanej iter() zob https://docs.python.org/3/library/functions.html#iter
        # return (i for i in (self.tytuł, self.ocena, self.długość))    # lub generator
        # który znamy z poprzednich zajęć

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __len__(self):                  # Tu uwaga: mamy iter, czyli Film jest trochę sekwencją, a len nie podaje długości tej sekwencji!
        return self.długość

    def __lt__(self, other):
        if self.ocena == other.ocena:
            return len(self) < len(other)
        return self.ocena < other.ocena


In [105]:
japoński = Film("Ikiru", 9, 143)
amerykański = Film("Lot nad kukułczym gniazdem", 10, 133)
polski = Film("Przypadek", 10, 114)

Dzięki `__repr__` mamy eleganckie wypisywanie, przydatne np. w debuggowaniu.

In [106]:
polski

Film:('Przypadek', 10, 114)

Zdefiniowanie `__len__` sprawia, że każdy będzie mógł wywołać `len(obj)`. Zauważmy, że będziemy traktować Film, jako obiekt iterowalny, zatem intuicyjnie po `len` spodziewalibyśmy się liczby elementów, po których iterujemy. Niech stanowi to przestrogę, przy korzystaniu z importowanych klas.

In [108]:
len(amerykański)  # mamy rozmiar obiektu - długość filmu

133

Funkcje `__iter__` sprawiają, że obiekt traktujemy podobnie jak inne w Pythonie...

Możemy po nim iterować

In [109]:
for i in amerykański: # dzięki __iter__ możemy iterować po obiekcie
    print(i)

Lot nad kukułczym gniazdem
10
133


Konwertować do standardowych kolekcji

In [110]:
tuple(japoński)

('Ikiru', 9, 143)

In [111]:
list(polski)

['Przypadek', 10, 114]

In [112]:
set(amerykański)

{10, 133, 'Lot nad kukułczym gniazdem'}

Możemy też nasze obiekty przechowywać w standardowych kolekcjach, a dzięki `__lt__` sortować te kolekcje.
Bez `__lt__` pojawi się `TypeError: '<' not supported between instances of 'Film' and 'Film'`

In [113]:
l = [japoński, amerykański, polski]
l.sort()    # dzięki __lt__ możemy sortować nasze obiekty
l

[Film:('Ikiru', 9, 143),
 Film:('Przypadek', 10, 114),
 Film:('Lot nad kukułczym gniazdem', 10, 133)]

Zauważmy, że nie mamy jeszcze wszystkich typowych możliwości

In [114]:
set(l)

TypeError: unhashable type: 'Film'

#### 1.1.2. Film v.1
Aby utworzyć zbiór filmów, obiekt Film musi być hashowalny. Musimy zaimplementować metodę `__hash__` (`__eq__` już mamy) i dopilnować, że instancje Film są niezmienne. Skorzystamy zatem z dekoratora `@property`. Metoda getter jest nazwana jak publiczna właściwość, którą eksponuje. Zatem każda metoda korzystająca z `self.atrybut` (jak np `__iter__`) może pozostać tak jak jest.

In [115]:
class Film:
    def __init__(self, tytuł, ocena, długość):
        self.__tytuł = tytuł
        self.__ocena = ocena
        self.__długość = długość

    @property                   # dodaję property dla atrybutów, które teraz są __atrybutami
    def tytuł(self):
        return self.__tytuł

    @property
    def ocena(self):
        return self.__ocena

    @ property
    def długość(self):
        return self.__długość

    def __hash__(self):         # dodaję metodę dunder hash
        return hash(tuple(self))

    # RESZTA PONIŻEJ POZOSTAJE BEZ ZMIAN
    def __repr__(self):
        return "Film:"+str(self)

    def __str__(self):
        return str(tuple(self))

    def __iter__(self):
        return iter((self.tytuł, self.ocena, self.długość))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __len__(self):
        return self.długość

    def __lt__(self, other):
        if self.ocena == other.ocena:
            return len(self) < len(other)
        return self.ocena < other.ocena

In [116]:
japoński = Film("Ikiru", 9, 143)
amerykański = Film("Lot nad kukułczym gniazdem", 10, 133)
polski = Film("Przypadek", 10, 114)
l = [japoński, amerykański, polski]
l.sort()    # dzięki __lt__ możemy sortować nasze obiekty
l

[Film:('Ikiru', 9, 143),
 Film:('Przypadek', 10, 114),
 Film:('Lot nad kukułczym gniazdem', 10, 133)]

In [117]:
set(l)

{Film:('Ikiru', 9, 143),
 Film:('Przypadek', 10, 114),
 Film:('Lot nad kukułczym gniazdem', 10, 133)}

Zatem czy mamy już wszystko? Zobaczmy może, czy nasz Film można traktować bezpośrednio jako sekwencję.

In [118]:
l = list(polski)
l[1]

10

In [119]:
polski[1]

TypeError: 'Film' object is not subscriptable

#### 1.1.3. Film v.2
Aby mieć dostęp do elementów w naszej klasie przez `[indeks]`, musimy zdefiniować metodę `__getitem__`

In [120]:
class Film:
    def __init__(self, tytuł, ocena, długość):
        self.__tytuł = tytuł
        self.__ocena = ocena
        self.__długość = długość

    def __getitem__(self, index):
        return tuple(self)[index]

    # RESZTA PONIŻEJ POZOSTAJE BEZ ZMIAN

    @property
    def tytuł(self):
        return self.__tytuł

    @property
    def ocena(self):
        return self.__ocena

    @ property
    def długość(self):
        return self.__długość

    def __hash__(self):
        return hash(tuple(self))

    def __repr__(self):
        return "Film:"+str(self)

    def __str__(self):
        return str(tuple(self))

    def __iter__(self):
        return iter((self.tytuł, self.ocena, self.długość))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __len__(self):
        return self.długość

    def __lt__(self, other):
        if self.ocena == other.ocena:
            return len(self) < len(other)
        return self.ocena < other.ocena

In [121]:
polski = Film("Przypadek", 10, 114)
polski[1]

10

In [122]:
polski[1:]

(10, 114)

Zauważmy, że działa nawet slicing. Nasza implementacja sprawia jednak, że zwracana jest krotka, a nie obiekt Film...

#### Podsumowując
Python do znajdowania atrybutów obiektów używa wiązań dynamicznych, które umożliwiają pracę z instancjami bez względu na ich typ. `Obj.name` zadziała na dowolnym `obj`, który ma `name` (kacze typowanie). Programiści Pythona często zamiast dziedziczyć po jakimś obiekcie, tworzą nowy niepowiązany obiekt, który wygląda i zachowuje się jak on. Tak było w omawianym przez nas przykładzie -  nasz Film byl iterowalny, ale nie dziedziczył po żadnej klasie `Iterable`.

### 1.2 Dziedziczenie

#### 1.2.1. Unikanie dziedziczenia - zamiast klas zastosuj funkcje
szczególnie w prostych przypadkach z jedną metodą. Rozważmy przypadek, gdzie różni widzowie zapamiętują różne aspekty filmu.

In [123]:
class Widz:
    def opowiedz(self, film):
        ciekawostki = []
        for el in film:
            if self.zapamiętuję(el):
                ciekawostki.append(el)
        return ciekawostki

    def zapamiętuję(self, el):
        return True

class WidzMatematyk(Widz):              # WidzMatematyk dziedziczy po Widz
    def zapamiętuję(self, el):
        return type(el) == int          # zapamiętuje tylko liczbowe atrybuty

widz = WidzMatematyk()
opis = widz.opowiedz(japoński)
opis

[9, 143]

Tymczasem rozwiązanie z funkcjami w takim przypadku jest prostsze i równie elastyczne.

In [124]:
def opowiedz(film, zapamiętuję):
    ciekawostki = []
    for el in film:
        if zapamiętuję(el):
           ciekawostki.append(el)
    return ciekawostki

def zapamiętuję_mat(el):                        # jedna polimorficzna funkcja z przykładu wyżej
    return type(el) == int

opis = opowiedz(japoński, zapamiętuję_mat)      # wybraną funkcję przekazuję jako argument
opis

[9, 143]

Podobnie dla tzw. zaczepów - metoda `sort()` pobiera opcjonalny argument `key`, któremu możemy przypisać funkcję (zob. materiały Python funkcyjny) `sort(key=len)`. W innych językach można oczekiwać, że zaczepy będą zdefiniowane przez klasę abstrakcyjną.

#### 1.2.2. Unikanie dziedziczenia przez stosowanie kompozycji
Załóżmy, że naszym zadaniem jest implementacja zbioru filmów do obejrzenia. W zamierzchłych czasach były one na nośnikach dvd i odkładając jeden film na drugi ... tworzył się stos. Bardzo szybko naszą implementację możemy otrzymać dziedzicząc po list.

In [125]:
class KolejkaOglądania(list):   # nasza kolejka dziedziczy po list
    def dodaj(self, el):
        self.append(el)

In [126]:
k = KolejkaOglądania()
k.dodaj(japoński)
k.dodaj(amerykański)
k.dodaj(polski)
k.pop()             # GANGNAM STYLE

Film:('Przypadek', 10, 114)

I oglądamy pierwszy film z góry. Nasze rozwiązanie oferuje też wiele więcej możliwości niż zwykły stos. Możemy np. sortować nasze filmy.

In [127]:
k.sort()
k

[Film:('Ikiru', 9, 143), Film:('Lot nad kukułczym gniazdem', 10, 133)]

In [128]:
k.sort(key=len)
k

[Film:('Lot nad kukułczym gniazdem', 10, 133), Film:('Ikiru', 9, 143)]

Bug czy Feature?
Nieoczekiwanie nasza KolejkaOglądania pozwala nam sortować filmy, co może być przydatne na platformach streamingowych.
Z drugiej strony niektórzy użytkownicy mogą uznać za dziwne, że model stosu filmów ma możliwość sortowania.
Lepszym podejściem jest **kompozycja**. Zamiast dziedziczyć po liście, stworzyć niezależną klasę, która ma w sobie listę.
Albo bardziej ogólnie:

In [129]:
class KolejkaOglądania:
    def __init__(self, container=None):
        if container is None:
            self._elementy = list()         # domyślnie trzymamy listę
        else:
            self._elementy = container

    def dodaj(self, el):
        self._elementy.append(el)

    def obejrzyj(self):
        return self._elementy.pop()

    def __len__(self):
        return len(self._elementy)

Nie korzystamy wprost z odziedziczonej implementacji (jw. wykonując `k.pop()`). Tylko oferujemy `obejrzyj`, w którym ukryty jest sposób dostępu i pobieranie z wewnętrznej struktury. Możemy łatwo podmienić wewnętrzną kolekcję z której korzystamy na inną, np. kolejkę.

In [130]:
import collections
k = KolejkaOglądania(container=collections.deque())
k.dodaj(japoński)
k.dodaj(polski)
k.obejrzyj()

Film:('Przypadek', 10, 114)

#### 1.2.3. Gęsie typowanie i abstrakcyjne klasy bazowe (klasy ABC)

Kacze typowanie może prowadzić do przypadkowych podobieństw:
`class Artist: def draw(self):...` (rysuj), `class Gunslinger: def draw(self):...` (wyciągnij), `class Loterry: def draw(self):...` (losuj).
Gęsie typowanie stosujmy jako uzupełnienie kaczego typowania. Obejmuje ono:
- tworzenie podklasy z abstrakcyjnej klasy bazowej (ABC), aby było jasne, że implementujemy wcześniej zdefiniowany interfejs
- sprawdzenie typów w czasie wykonywania programu przy użyciu klas ABC

#### Złote myśli
Scott Mayer w "More effective C++" pisze "wszystkie klasy, po których można dziedziczyć, powinny być abstrakcyjne"
Alex Martelli, cytowany w "Fluent Python", twierdzi "możemy .. definiować swoje abstrakcyjne klasy bazowe, ale odradzam to wszystkim .. tak samo, jak odradzałbym definiowanie swoich własnych niestandardowych metaklas"

Zobaczmy, jak tam nasza klasa Film. O klasach ABC można doczytać tu https://docs.python.org/3/library/collections.abc.html

In [131]:
from collections import abc
isinstance(polski, abc.Sized)

True

Sized rozpoznaje Film jako podklasę, dzięki implementacji `__len__`

In [132]:
(isinstance(polski, abc.Iterable), isinstance(polski, abc.Hashable), isinstance(polski, abc.Container), isinstance(polski, abc.Collection))

(True, True, False, False)

Dla ostatnich dwóch brakuje nam implementacji `__contains__`.

## 2. Wielokrotne dziedziczenie

to możliwość wyprowadzenia klasy z więcej niż jednej klasy bazowej. C++ je obsługuje, Java i C# nie. Python tak.

Poniżej mamy przykład tzw. diamentowego dziedziczenia. Konstruując obiekt, chcemy wypisać wszystkie jego cechy. Tymczasem, startując z dołu hierarchii, dwukrotnie uruchomimy opis z klasy na szczycie hierarchii.

In [133]:
class Kinowa:
    def __init__(self):
        print("Kinowa")                             # do śledzenia kolejności wywołań

class Komedia(Kinowa):
    def __init__(self):
        print("Komedia")
        Kinowa.__init__(self)

class Romantyczna(Kinowa):
    def __init__(self):
        print("Romantyczna")
        Kinowa.__init__(self)

class KomediaRomantyczna(Komedia, Romantyczna):     # przykład wielokrotnego dziedziczenia
    def __init__(self):
        print("KR")
        Komedia.__init__(self)
        Romantyczna.__init__(self)

kr = KomediaRomantyczna()

KR
Komedia
Kinowa
Romantyczna
Kinowa


Korzystając z `super()` udaje się nam uzyskać to, co zamierzaliśmy.

In [134]:
class Kinowa:
    def __init__(self):
        print("Kinowa")

class Komedia(Kinowa):
    def __init__(self):
        print("Komedia")
        super().__init__()          # zamiast Kinowa.__init__()

class Romantyczna(Kinowa):
    def __init__(self):
        print("Romantyczna")
        super().__init__()

class KomediaRomantyczna(Komedia, Romantyczna):
    def __init__(self):
        print("KR")
        super().__init__()

kr = KomediaRomantyczna()

KR
Komedia
Romantyczna
Kinowa


Dodatkowo napis "Kinowa" wypisał się na samym końcu. Jak w takim razie ustalana jest kolejność?

## 3. Kolejność odszukiwania metod - Method resolution order - MRO

Znamy już z poprzednich materiałów przydatne polecenie `dir`

In [135]:
dir(polski)

['_Film__długość',
 '_Film__ocena',
 '_Film__tytuł',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'długość',
 'ocena',
 'tytuł']

Jak widzimy, nie mamy na tej liście `__mro__`. A jednak możemy go uruchomić.

In [136]:
Film.__mro__                # tak

(__main__.Film, object)

In [137]:
KomediaRomantyczna.mro()    # lub tak

[__main__.KomediaRomantyczna,
 __main__.Komedia,
 __main__.Romantyczna,
 __main__.Kinowa,
 object]

In [138]:
Romantyczna.mro()

[__main__.Romantyczna, __main__.Kinowa, object]

MRO określa kolejność wyszukiwania. Za każdym razem, gdy szukasz atrybutu w instancji lub klasie, każda klasa w MRO jest sprawdzana w podanej kolejności.
Wyszukiwanie zatrzymuje się po pierwszym dopasowaniu.
Zauważmy, że jak wyszło we wszystkich powyższych przykładach, wszystko dziedziczy po object.

Współdzielone dziedziczenie wielokrotne - wszystkie klasy umieszczane są na liście MRO tak, aby:
- klasa potomna jest przed każdym z rodziców
- jeśli klasa ma wielu rodziców, sprawdzani są oni w tej samej kolejności, co zapisani na liście dziedziczenia dziecka

Kolejność określana jest zgodnie z algorytmem linearyzacji C3.

In [139]:
class Serial: pass
class Telenowela(Serial): pass
class Wenezuelska(Serial, Telenowela): pass

TypeError: Cannot create a consistent method resolution
order (MRO) for bases Serial, Telenowela

Dla takiej hierarchii Python zgłosił nam błąd. Tymczasem jeśli zamienimy kolejność na liście dziedziczenia klasy Wenezuelska, już będzie ok:

In [140]:
class Serial: pass
class Telenowela(Serial): pass
class Wenezuelska(Telenowela, Serial): pass

Podobnie jak w wyżej omówionym przypadku konstruktorów, nie stosowanie `super()` prowadzi do nieintuicyjnego działania.

In [141]:
class Kinowa:
    def wypisz(self):
        print("Kinowa")

class Komedia(Kinowa):
    def wypisz(self):
        print("Komedia")
        super().wypisz()

class Romantyczna(Kinowa):
    def wypisz(self):
        print("Romantyczna")

class KomediaRomantyczna(Komedia, Romantyczna):
    pass

kr = KomediaRomantyczna()
kr.wypisz() # co się stanie?

Komedia
Romantyczna


Nawet dobry opis. Ale dlaczego taki? Prześledźmy.

In [142]:
KomediaRomantyczna.mro()

[__main__.KomediaRomantyczna,
 __main__.Komedia,
 __main__.Romantyczna,
 __main__.Kinowa,
 object]

Przez dziedziczenie `kr` aktywuję `wypisz()` w klasie `Komedia`. Tam z kolei w implementacji `wypisz()` jest odwołanie `super()`.
Zauważmy, że nie odnosi się ono do `wypisz()` z klasy `Kinowa`, która jest nadklasą `Komedia`.
Odniesie się ono do następnej pozycji w liście MRO, w `Romantyczna` jest metoda `wypisz()` i to ona zostanie wywołana.

Analogicznie do konstruktorów, jeśli chcemy wykonać wszystkie funkcje z nadklas jednokrotnie, wszędzie użyjemy super.

In [143]:
class Kinowa:
    def wypisz(self):
        print("Kinowa")

class Komedia(Kinowa):
    def wypisz(self):
        print("Komedia")
        super().wypisz()

class Romantyczna(Kinowa):
    def wypisz(self):
        print("Romantyczna")
        super().wypisz()

class KomediaRomantyczna(Komedia, Romantyczna):
    pass

kr = KomediaRomantyczna()
kr.wypisz() # co się stanie?

Komedia
Romantyczna
Kinowa


## 4. Klasy domieszki

#### 4.1. Wprowadzenie
Klasa domieszka to klasa, która modyfikuje lub rozszerza funkcjonalność innych klas.

Wyobraźmy sobie, że mamy klasy z wielokrotnie ponownie używanym dźwiękiem do filmów (komentarze w kodzie to ciekawostki filmowe, nie programistyczne)

In [144]:
class KtośSpada:        # zob https://pl.wikipedia.org/wiki/Krzyk_Wilhelma
    def dźwięk(self):   # wykorzystywany w wielu filmach, grach
        return 'aaaah'  # potem też taki "easter egg" dźwiękowców filmowych

class PojedynekRewolwerowy: # autor: Morricone, reż: Sergio Leone, "The good, the bad and the ugly"
    def dźwięk(self):       # poniższa transkrypcja działa :) https://www.reddit.com/r/NoStupidQuestions/comments/f6jomc/whats_the_song_called_that_is_usually_used_in/
        return 'wolololol waw waw waw ayayayayaya waw waw waw'      # Muzyka Morricone jeszcze lepsza jest w "Pewnego razu na Dzikim Zachodzie" Sergio Leone :)

class StrasznaScena:    # autor: Bernard Herrmann, reż: Hitchcock, "Psychoza"
    def dźwięk(self):
        return 'hi hi hi hi hi'

class Niepokój:         # autor: Lynch, reż: Lynch, "Eraserhead"
    def dźwięk(self):   # tych dźwięków akurat chyba nikt nie użył ponownie
        return 'aaa aaaaaaa aaaaaa'

Oraz zupełnie niepowiązane klasy, jak np. Kosiarka.

In [145]:
class Kosiarka:
    def dźwięk(self):                # jedna z wielu metod, zupełnie inna natura niż w klasach opisanych wyżej
        return 'wrrrr'
    def koś(self, lokalizacja): pass
    def uzupełnij_paliwo(self): pass # i wiele innych

Zauważmy, że KtośSpada i Kosiarka to klasy ze sobą zupełnie niepowiązane, nie ma między nimi relacji dziedziczenia, stosują różne metody. Cechą wspólną jest metoda `dźwięk()`. Skonstruujemy teraz klasy domieszki implementujące brakujące funkcjonalności - pogłośń i powtórz.
Konwencja nazewnictwa zaleca dołączyć człon `Mixin` do klas domieszek (wcześniej konsekwentnie używaliśmy polskich nazw, więc przy nich zostańmy)

In [146]:
class PogłośńMixin:
    def dźwięk(self):
        return super().dźwięk().upper()

class PowtórzMixin:
    def dźwięk(self):
        return 2*(super().dźwięk() + " ")

Klasa taka sama w sobie nie działa (deleguje do nieistniejącej klasy nadrzędnej).

In [147]:
powtarzarka = PowtórzMixin()
powtarzarka.dźwięk()

AttributeError: 'super' object has no attribute 'dźwięk'

Możemy za to za ich pomocą implementować nową funkcjonalność

In [148]:
class GłośneSpadanie(PogłośńMixin, KtośSpada): pass
class StaraKosiarka(PowtórzMixin, Kosiarka): pass
class DługiGłośnyPojedynek(PowtórzMixin, PogłośńMixin, PojedynekRewolwerowy): pass

I mamy:

In [149]:
upadek = GłośneSpadanie()
upadek.dźwięk()

'AAAAH'

In [150]:
nie_chce_zapalić = StaraKosiarka()
nie_chce_zapalić.dźwięk()

'wrrrr wrrrr '

In [151]:
klimatyczna_scena = DługiGłośnyPojedynek()
klimatyczna_scena.dźwięk()

'WOLOLOLOL WAW WAW WAW AYAYAYAYAYA WAW WAW WAW WOLOLOLOL WAW WAW WAW AYAYAYAYAYA WAW WAW WAW '

Spójrzmy teraz na MRO naszej klasy

In [152]:
DługiGłośnyPojedynek.mro()

[__main__.DługiGłośnyPojedynek,
 __main__.PowtórzMixin,
 __main__.PogłośńMixin,
 __main__.PojedynekRewolwerowy,
 object]

#### 4.2. Przykłady zastosowań

Spróbujmy przeedytować dźwięk na pierwszym filmie dźwiękowym.

In [153]:
class DługiGłośnyFilm(PowtórzMixin, PogłośńMixin, Film): pass
dźwiękowy = DługiGłośnyFilm("The jazz singer", 7, 88)
dźwiękowy.dźwięk()

AttributeError: 'super' object has no attribute 'dźwięk'

Rzeczywiście, klasa film nie ma przecież metody `dźwięk`. Możemy rozwiązać ten problem za pomocą wspólnego rodzica naszych domieszek. Jeśli nie będzie dźwięku, zwrócimy pusty napis.

In [154]:
class DźwiękMixin:
    def dźwięk(self):
        try:
            ret = super().dźwięk()
        except AttributeError:          # gdy nie znajdziemy metody dźwięk()
            ret = ''
        return ret

# POZOSTAŁE METODY ZOSTAJĄ BEZ ZMIAN
class PogłośńMixin(DźwiękMixin):
    def dźwięk(self):
        return super().dźwięk().upper()

class PowtórzMixin(DźwiękMixin):
    def dźwięk(self):
        return 2*(super().dźwięk() + " ")

In [155]:
class DługiGłośnyFilm(PowtórzMixin, PogłośńMixin, Film): pass

In [156]:
dźwiękowy = DługiGłośnyFilm("The jazz singer", 7, 88)
dźwiękowy.dźwięk()

'  '

Udało nam się. Nie tylko nie wystąpił błąd, ale jak nie ma zdefiniowanego dźwięku, mamy transkrypcję ciszy.

Przyjrzyjmy się jeszcze kolejności wywołań i przywołajmy poprzednią klasę `DługiGłośnyPojedynek`.

In [157]:
class DługiGłośnyPojedynek(PowtórzMixin, PogłośńMixin, PojedynekRewolwerowy): pass
print(DługiGłośnyFilm.mro())
DługiGłośnyPojedynek.mro()

[<class '__main__.DługiGłośnyFilm'>, <class '__main__.PowtórzMixin'>, <class '__main__.PogłośńMixin'>, <class '__main__.DźwiękMixin'>, <class '__main__.Film'>, <class 'object'>]


[__main__.DługiGłośnyPojedynek,
 __main__.PowtórzMixin,
 __main__.PogłośńMixin,
 __main__.DźwiękMixin,
 __main__.PojedynekRewolwerowy,
 object]

Zauważmy, że klasa pod `obiect` czyli `Film` / `PojedynekRewolwerowy` są w MRO po `DźwiękMixin`.
Zatem, gdyby w `DźwiękMixin` zwracać po prostu pusty napis, `dźwięk` z `PojedynekRewolwerowy` zostałby przez niego zastąpiony.
Dlatego w naszej implementacji użyliśmy `super().dźwięk()` i działa:

In [158]:
klimatyczna_scena = DługiGłośnyPojedynek()
klimatyczna_scena.dźwięk()

'WOLOLOLOL WAW WAW WAW AYAYAYAYAYA WAW WAW WAW WOLOLOLOL WAW WAW WAW AYAYAYAYAYA WAW WAW WAW '

### Do zapamiętania:
- W Javie dopuszczalne jest wielokrotne dziedziczenie interfejsów, w podobnym duchu korzystaj z wielodziedziczenia w Pythonie
- klasy ABC mogą mieć rolę interfejsu, mogą też mieć rolę klas domieszkowych

Przykłady klas domieszkowych:
- dla zarządzania wątkami w socketserver - ThreadingMixIn i ForkingMixIn
- dla widoków w Django (framework do aplikacji internetowych) - TemplateResponseMixin, ContextMixin, ...
- dla drzew - np. w module filogenetycznym w biopythonie - Bio.Phylo.BaseTree.TreeMixin
...

## 5. Różne

#### 5.0. A teraz coś z zupełnie innej beczki ...

##### 5.0.1. Metoda `__new__`

In [159]:
class Statysta:
    def __new__(cls):
        print("new")
        return super().__new__(cls)

    def __init__(self):
        print("init")

Statysta()

new
init


Wywołanie `Statysta()` wywołuje metodę `__new__` z klasy Statysta, ta z kolei (jak widać) wywołuje `__new__` z nadklasy, z nazwą klasy jako parametrem, co (dokładniej: w klasie object) powoduje stworzenie "pustej" instancji.

Ta z kolei następnie jest przekazywana do `__init__` z klasy Statysta.

Metoda `__new__` to metoda statyczna, jej pierwszy argument `cls` to referencja do klasy, z której jest wywoływana (zwykle jest to klasa potomna klasy, w której `__new__` jest zdefiniowana).

Nadpisanie `__new__` może służyć temu, by zapobiec tworzeniu instancji w określonych okolicznościach, do stworzenia instancji (trochę ;) innej klasy, albo w ogóle zwrócenia istniejącego obiektu - chociaż w tym przypadku wywołane zostanie na nim `__init__` - nie róbmy tego ;)

##### 5.0.2. `@classmethod` i `@staticmethod`

Dekorator `@classmethod` zmienia sposób wywołania metody, aby jako pierwszy argument otrzymywała samą klasę (oznaczaną zwykle jako `cls`) zamiast instancji (oznaczaną `self`).

Najczęstsze zastosowanie tej metody to alternatywne konstruktory (`@classmethod` pozwala rozwiązać problemy, które w innych językach można rozwiązać korzystając z polimorfizmu konstruktora). W Pythonie polimorfizm obsługują nie tylko obiekty, ale też klasy.

In [160]:
class LicznikInstancjiMixin:
    __licznik = 0
    def __init__(self):
        self.zwieksz_licznik(1)

    @classmethod
    def zwieksz_licznik(cls, ile):
        cls.__licznik += ile

    @classmethod
    def licznik(cls):
        return cls.__licznik

In [161]:
class KillBill(LicznikInstancjiMixin): pass

class OjciecChrzestny(LicznikInstancjiMixin): pass

In [162]:
p1 = KillBill()
print(p1.licznik(), KillBill.licznik())
p2 = KillBill()
print(p2.licznik(), KillBill.licznik())

d1, d2, d3 = OjciecChrzestny(), OjciecChrzestny(), OjciecChrzestny()
print("Ojciec Chrzestny", d1.licznik(), "z", OjciecChrzestny.licznik())
print("Kill Bill", p2.licznik(), "z", KillBill.licznik())

1 1
2 2
Ojciec Chrzestny 3 z 3
Kill Bill 2 z 2


Jak widać, liczniki instancji są niezależne w każdej z klas.

Dekorator `@staticmethod` zmienia metodę, aby nie otrzymywała żadnego specjalnego pierwszego argumentu. Taka metoda nie ma dostępu do atrybutów klasowych i atrybutów instancji - nie może ona mieć parametrów `cls` (charakterystycznych dla metod klasowych) i `self` (związanych z instancja klasy).

Przeanalizujmy powyższe rozwiązanie, gdyby zamiast `@classmethod` implementować `@staticmethod`. Metody `zwieksz_licznik` i `licznik` nie mogą mieć parametru `cls`, zatem w najlepszym razie mogą używać tylko pełnej nazwy (jednej) klasy i siłą rzeczy taki licznik byłby wspólny:

In [163]:
class WspólnyLicznikInstancji:
    __licznik = 0
    def __init__(self):
        self.zwieksz_licznik(1)

    @staticmethod
    def zwieksz_licznik(ile):
        WspólnyLicznikInstancji.__licznik += ile

    @staticmethod
    def licznik():
        return WspólnyLicznikInstancji.__licznik

In [164]:
# TA CZĘŚĆ POZOSTAJE BEZ ZMIAN
class KillBill(WspólnyLicznikInstancji): pass

class OjciecChrzestny(WspólnyLicznikInstancji): pass

p1 = KillBill()
print(p1.licznik(), KillBill.licznik())
p2 = KillBill()
print(p2.licznik(), KillBill.licznik())

d1, d2, d3 = OjciecChrzestny(), OjciecChrzestny(), OjciecChrzestny()
print("Ojciec Chrzestny", d1.licznik(), "z", OjciecChrzestny.licznik())
print("Kill Bill", p2.licznik(), "z", KillBill.licznik())

1 1
2 2
Ojciec Chrzestny 5 z 5
Kill Bill 5 z 5


I mamy "Ojca Chrzestnego 5" ...

#### 5.1. Dyspozycja (ang. dispatch) na podstawie typu
Zamiast nieeleganckiej serii if-else i sprawdzania isinstance(), możemy stworzyć klasę, która oferuje różne metody dla różnych obiektów. Zobacz poniżej:

In [165]:
class Dispatcher:
    def obsłuż(self, obj):
        for ty in type(obj).__mro__:    # wyszukuję po liście MRO odpowiedniej funkcji
            met = getattr(self, f'obsłuż_{ty.__name__}', None)
            if met:
                return met(obj)
        raise RuntimeError(f'Brak obsługi dla {obj}')

    def obsłuż_Kosiarka(self, obj):
        print("koszę")
    def obsłuż_KtośSpada(self, obj):
        print("spadam")
    def obsłuż_Film(self, obj):
        print(f"Oglądam {obj}")

dispatcher = Dispatcher()
dispatcher.obsłuż(KtośSpada())
dispatcher.obsłuż(StaraKosiarka())
dispatcher.obsłuż(japoński)

spadam
koszę
Oglądam ('Ikiru', 9, 143)


Zauważ, że wykorzystaliśmy MRO i dla `StaraKosiarka()` wywołała się metoda `obsłuż_Kosiarka`.

#### 5.2 Dekorator klasy
to funkcja, która pobiera klasę jako dane wejściowe i zwraca klasę jako dane wyjściowe.

##### 5.2.1. Dekorator klasy, jako alternatywa dla klasy domieszkowej
Dekorator klasy ma wiele zastosowań, tutaj przedstawimy alternatywne rozwiązanie dla klasy domieszkowej.
Funkcje `głośno` i `powtórz` zastąpią `PogłośńMixin` i `PowtórzMixin`, jako dekorator klasy `Kosiarka`.

In [166]:
def głośno(klasa):
    oryg_dźwięk = klasa.dźwięk
    def dźwięk(self):
        return oryg_dźwięk(self).upper()
    klasa.dźwięk = dźwięk
    return klasa

def powtórz(klasa):
    oryg_dźwięk = klasa.dźwięk
    def dźwięk(self):
        return 2*(oryg_dźwięk(self) + " ")
    klasa.dźwięk = dźwięk
    return klasa

@powtórz
@głośno
class Kosiarka:
    def dźwięk(self):
        return 'wrrr'

In [167]:
k = Kosiarka()
k.dźwięk()

'WRRR WRRR '

##### 5.2.2 A teraz użyjemy dekoratora klasy do rejestracji klas.
Przykładem zastosowania, może być szereg implementacji dekoderów. I chcemy wybrać klasę do obsługi naszego typu plików.
Tutaj przeniesiemy to na grunt filmowy. Mamy szereg klas reżyserów. Każdy ma `cechy_filmów`, które definiują, jak wpływa na Widza w metodzie `wpłyń`.
Załóżmy, że chcemy wybrać klasę reżysera, podając cechę filmu, aby dostać oczekiwany wpływ na widza.

In [168]:
_rejestr = {}                                   # słownik klucz: cecha wartość: klasa reżysera
def rejestr_reżyserów(klasa):                   # klasa reżyserów
    for cecha in klasa.cechy_filmów:
        _rejestr[cecha] = klasa                 # w rejestrze zapisuję
    return klasa

def wybierz_reżysera(cecha):
    return _rejestr[cecha]()                    # tworzy obiekt klasy pobranej z rejestru

Przed każdą klasą musimy napisać dekorator (niebawem zobaczymy, jak to zrobić w jednej klasie)

In [169]:
@rejestr_reżyserów
class Capra:
    cechy_filmów = ['optymizm', 'chce się żyć']
    def wpłyń(self, widz): pass                     # pokrzepia na duchu

@rejestr_reżyserów
class Hitchcock:
    cechy_filmów = ['emocjonujący', 'potrafi przestraszyć']
    def wpłyń(self, widz): pass                     # powoduje katharsis

In [170]:
reżyser = wybierz_reżysera('optymizm')

In [171]:
reżyser

I już wiemy, że wywołanie `reżyser.wpłyń(obj)` zadziała według oczekiwanej cechy "optymizm".

#### 5.3. Nadzorowane dziedziczenie
Klasa nadrzędna może wykonywać akcje w imieniu swoich podklas dzięki `__init_subclass__`.

Zamast funkcji `rejestr_reżyserów` użytej jako dekoratora klas (Capra, Hitchcock). Tworzymy klasę `Reżyser` po której będą dziedziczyć reżyserzy (Capra, Hitchcock, ...) i zastosujemy `__init_subclass__()`.

In [2]:
class Reżyser:
    _rejestr = {}
    @classmethod
    def __init_subclass__(klasa):
        for cecha in klasa.cechy_filmów:
            Reżyser._rejestr[cecha] = klasa

Funkcja `wybierz_reżysera` podobnie tworzy klasę z rejestru.

In [3]:
def wybierz_reżysera(cecha):
    return Reżyser._rejestr[cecha]()

Klasy zamiast dekoratora przed klasą, dziedziczą po `Reżyser`.

In [4]:
class Capra(Reżyser):
    cechy_filmów = ['optymizm', 'chce się zyć']
    def wpłyń(self, widz): pass                     # pokrzepia na duchu

class Hitchcock(Reżyser):
    cechy_filmów = ['emocjonujący', 'potrafi przestraszyć']
    def oglądaj(self, widz): pass                   # powoduje katharsis

Wynik będzie taki sam.

In [6]:
reżyser = wybierz_reżysera('optymizm')
reżyser

#### 5.4. Zmniejszenie wykorzystania pamięci za pomocą `__slots__`

Standardowo, stan powiązany z instancją każdej klasy przechowywany jest w słowniku, który jest dostępny jako atrybut instancji (`__dict__`). Jeśli tworzymy bardzo dużą liczbę instancji, to mamy bardzo dużą liczbę słowników, a więc obciąża to pamięć. Gdy zestaw nazw atrybutów jest stały (a zwykle tak jest), można te nazwy zapisać w specjalnej zmiennej klasy o nazwie `__slots__`. Dzięki temu dostęp do nich staje się szybszy, bo nie przechodzi przez `__dict__`. A dokładniej, atrybut `__dict__` w ogóle nie jest tworzony dla instancji takiej klasy, co powoduje też, że instancja zużywa mniej pamięci.

In [177]:
class Montaż:
    __slots__ = ('autor', 'metoda')
    def __init__(self, autor, sposób):
        self.autor = autor
        self.metoda = sposób

Uwagi:
- jeśli klasa dziedziczy po klasie gdzie jest `__slots__`, to również musi zdefiniować `__slots__`, ponieważ dziedziczony atrybut jest ignorowany przez interpreter. Gdy o tym zapomnimy, nasze rozwiązanie będzie działać wolniej i zużyje więcej pamięci niż w przypadku bez `__slots__` w żadnej z klas.
- w dziedziczeniu wielokrotnym, jeśli wiele klas bazowych ma niepuste `__slots__`, otrzymamy `TypeError`
- pamiętaj, że nie ma już `__dict__`, z którego zewnętrzne biblioteki i narzędzia do obsługi obiektów mogą korzystać
- użycie slots jest dość szczególne, ma znaczenie gdy obsługujemy miliony instancji, w większości przypadków użycie pandas może być najlepszą opcją
- oprócz wymienionej na początku literatury - zainteresowanych odsyłam do [https://stackoverflow.com/questions/472000/usage-of-slots](https://stackoverflow.com/questions/472000/usage-of-slots)

## 6. Metaklasy

Kiedy definiujesz klasę w Pythonie, sama definicja staje się obiektem.
Tworzenie obiektu klasy kontrolowane jest przez specjalny rodzaj klasy zwany metaklasą.

In [178]:
class Scenariusz: pass

Dopiero zaczynamy. Nie mamy jeszcze, żadnego obiektu danej klasy, ale możemy zrobić to:

In [179]:
isinstance(Scenariusz, object)

True

Skoro Scenariusz istnieje, coś musiało go stworzyć. Zobaczmy:

In [180]:
Scenariusz.__class__

type

Kiedy klasa definiowana jest przez `class` dzieją się następujące rzeczy.

In [181]:
namespace = type.__prepare__('Scenariusz', ())      # utwórz przestrzeń nazw klas
exec('''pass''', globals(), namespace)              # wykonaj treść klasy
Scenariusz = type('Scenariusz', (), namespace)      # utwórz ostateczny obiekt klasy

#### 6.1. Przykład 1

Wiemy już, że przeciążanie nie działa w Pythonie. Zduplikowane definicje działają tak, że druga zastępuje pierwszą. Chcemy wyłapać takie przypadki w naszym kodzie. Zastosujemy metaklasę.

In [182]:
class Słownik(dict):                            # Słownik dla przestrzeni nazw klasy; dziedziczy po dict
    def __setitem__(self, key, value):          # dodanie do naszego Słownika
        if key in self:                         # sprawdza, czy był element, jak tak to błąd
            raise AttributeError(f'{key} już zdefiniowany')
        super().__setitem__(key, value)         # jak nie to wywołuje standardową metodę słownika

Tworzymy teraz naszą Metaklasę. Dziedziczy ona po `type`. Tworzymy własną `__prepare__`, aby wykorzystać nasz `Słownik`, który będzie zgłaszał błąd, jeśli nazwa się powtórzy.

In [183]:
class BezDuplikacjiMeta(type):
    @classmethod
    def __prepare__(metacls, name, bases):
        return Słownik()

Teraz możemy zdefiniować jakąś klasę i wskazać jej metaklasę.

In [184]:
class Człowiek(metaclass=BezDuplikacjiMeta):
    pass

Metaklasę się dziedziczy. Klasy, gdzie nie wskazuje się dziedziczenia, dziedziczą po `object` metaklasę `type`.
Stworzymy teraz klasę `Aktor`, która dziedziczy po `Człowiek` i w ten sposób dotyczy jej metaklasa `BezDuplikacjiMeta`.

In [185]:
class Aktor(Człowiek):
    def graj(self): pass
    def graj(self, film): pass

AttributeError: graj już zdefiniowany

Hurra, mamy nasz błąd.

#### 6.2. Przykład 2

Wykorzystamy teraz metaklasę do przepisania zawartości przestrzeni nazw klasy przed utworzeniem obiektu klasy.
Pewne cechy klasy są ustalane w momencie definiowania klasy i nie mogą być później modyfikowane. Przykładem jest omówione wyżej `__slots__`.
Poniżej mamy metaklasę, która na podstawie sygnatury metody `__init__`, automatycznie ustawia atrybut `__slots__`.

In [187]:
import inspect

class SlotMeta(type):
    @staticmethod
    def __new__(metaklasa, nazwa_nowej_klasy, klasy_bazowe, atrybuty):
        if '__init__' in atrybuty:                                      # jeśli jest init tworzę slots
            sygnatura = inspect.signature(atrybuty['__init__'])
            __slots__ = tuple(sygnatura.parameters)[1:]
        else:
            __slots__ = ()                                              # jeśli nie to slots pusty
        atrybuty['__slots__'] = __slots__
        return super().__new__(metaklasa, nazwa_nowej_klasy, klasy_bazowe, atrybuty)

In [188]:
class Człowiek(metaclass=SlotMeta):
    pass

Teraz wszyscy dziedziczący po `Człowiek`, będą obsługiwani przez `SlotMeta`.

In [189]:
class Aktor(Człowiek):
    def __init__(self, imię, nazwisko, znane_języki):
        pass

Zobaczmy, `Aktor` teraz ma atrybut `slots`

In [190]:
dir(Aktor)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 'imię',
 'nazwisko',
 'znane_języki']

Możemy się do niego odwołać.

In [191]:
Aktor.__slots__

('imię', 'nazwisko', 'znane_języki')

In [192]:
class Statysta:
    def __init__(self, imię, nazwisko, znane_języki):
        pass

Tymczasem `Statysta` nie dziedziczy po `Człowiek`, czyli ma standardową metaklasę i nie ma `slots`

In [193]:
dir(Statysta)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

I jeśli spróbujemy się do niego dostać, zobaczymy błąd.

In [194]:
Statysta.__slots__

AttributeError: type object 'Statysta' has no attribute '__slots__'

##### Podsumowując:
Historycznie Metaklasy były używane do różnych zadań, które teraz są możliwe do zrealizowania za pomocą innych środków.
W książce "Efektywny Pyhon", wszystkie sposoby dotyczące metaklas, podpowiadają jak ich nie stosować:
- zamiast refaktoryzować atrybuty można użyć `@property`
- stosuj deskryptory
- wykorzystaj `__init_subclass__()`
- adnotacje atrybutów klas dodawaj za pomocą `__set_name__()` (tego już nie zdążyliśmy omówić)
- dla złożonych rozszerzeń klas wybieraj dekoratory klas zamiast metaklas (metaklas nie można łączyć ze sobą, a wiele dekoratorów klas można wykorzystać do rozszerzenia jednej klasy)

Na szczęście nasz ostatni przykład ze `__slots__` nie da się rozwiązać używając dekoratorów klas, czy za pomocą `__init_subclass__()`, ponieważ te funkcje działają na klasie już po jej utworzeniu.
